In [61]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium as fm

In [62]:
products_cols = ["ProductID","ProductName","SupplierID","CategoryID","UndiscountedUnitPrice",]
suppliers_cols = ["SupplierID","CompanyName","ContactName","ContactTitle","City","Region","Country"]
orders_cols = ["OrderID","OrderDate","RequiredDate","ShippedDate","ShipVia","Freight","ShipName","ShipCity","ShipRegion","ShipPostalCode","ShipCountry"]
categories_cols = ["CategoryID","CategoryName"]

products = pd.read_csv("../data/products.csv", usecols=products_cols, encoding='latin1')
categories = pd.read_csv("../data/categories.csv", usecols=categories_cols, encoding='latin1')
suppliers = pd.read_csv("../data/suppliers.csv", usecols=suppliers_cols, encoding='latin1')
orders = pd.read_csv("../data/orders.csv", usecols= orders_cols, encoding='latin1')
order_details = pd.read_csv("../data/order_details.csv")
shippers = pd.read_csv("../data/shippers.csv")


In [63]:
products.to_csv("products.csv", index=False)
suppliers.to_csv("suppliers.csv", index=False)
orders.to_csv("orders.csv", index=False)
order_details.to_csv("order_details.csv", index=False)
shippers.to_csv("shippers.csv", index=False)
categories.to_csv("categories.csv", index=False)

In [64]:
table_dict = {"products": products, "categories":categories,"suppliers": suppliers, "orders": orders, "order_details": order_details, "shippers": shippers}


for table_name, table in table_dict.items():
    print(f"{table_name} table has {table.shape[0]} rows and {table.shape[1]} columns")
    print(table.info())
    print("\n")

products table has 77 rows and 5 columns
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ProductID              77 non-null     int64 
 1   ProductName            77 non-null     object
 2   SupplierID             77 non-null     int64 
 3   CategoryID             77 non-null     int64 
 4   UndiscountedUnitPrice  77 non-null     object
dtypes: int64(3), object(2)
memory usage: 3.1+ KB
None


categories table has 8 rows and 2 columns
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CategoryID    8 non-null      int64 
 1   CategoryName  8 non-null      object
dtypes: int64(1), object(1)
memory usage: 260.0+ bytes
None


suppliers table has 29 rows and 7 columns
<class 'pandas.core.frame.Dat

In [65]:
display(products.head(3), categories.head(3),suppliers.head(3), orders.head(3), order_details.head(3), shippers.head(3))

,ProductID,ProductName,SupplierID,CategoryID,UndiscountedUnitPrice
0,1,Chai,1,1,$18.00
1,2,Chang,1,1,$19.00
2,3,Aniseed Syrup,1,2,$10.00


,CategoryID,CategoryName
0,1,Beverages
1,2,Condiments
2,3,Confections


,SupplierID,CompanyName,ContactName,ContactTitle,City,Region,Country
0,1,Exotic Liquids,Charlotte Cooper,Purchasing Manager,London,NaN,UK
1,2,New Orleans Cajun Delights,Shelley Burke,Order Administrator,New Orleans,LA,USA
2,3,Grandma Kelly's Homestead,Regina Murphy,Sales Representative,Ann Arbor,MI,USA


,OrderID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10389,1995-01-20,1995-02-17,1995-01-24,2,47.42,Bottom-Dollar Markets,Tsawassen,BC,T2F 8M4,Canada
1,10390,1995-01-23,1995-02-20,1995-01-26,1,126.38,Ernst Handel,Graz,NaN,8010,Austria
2,10391,1995-01-23,1995-02-20,1995-01-31,3,5.45,Drachenblut Delikatessen,Aachen,NaN,52066,Germany


,OrderID,ProductID,DiscountedUnitPrice,Quantity
0,10248,11,14.0,12
1,10248,42,9.8,10
2,10248,72,34.8,5


,ShipperID,CompanyName,Phone
0,1,Speedy Express,(503) 555-9831
1,2,United Package,(503) 555-3199
2,3,Federal Shipping,(503) 555-9931


## Data Preparation

In [66]:
products["UndiscountedUnitPrice"] = products["UndiscountedUnitPrice"].str.replace("$","").astype("float64")

orders["OrderDate"] = pd.to_datetime(orders["OrderDate"])
orders["ShippedDate"] = pd.to_datetime(orders["ShippedDate"])
orders['ShippingTime'] = (orders['ShippedDate'] - orders['OrderDate']).dt.days

orders.rename(columns={"ShipVia":"ShipperID"}, inplace=True)

In [67]:
suppliers_analysis = (
    products.merge(categories, on="CategoryID")
    .merge(suppliers, on="SupplierID")
    .merge(order_details, on="ProductID")
    .merge(orders, on="OrderID")
    .merge(shippers, on="ShipperID")
).head(3)

suppliers_analysis

,ProductID,ProductName,SupplierID,CategoryID,UndiscountedUnitPrice,CategoryName,CompanyName_x,ContactName,ContactTitle,City,...,ShipperID,Freight,ShipName,ShipCity,ShipRegion,ShipPostalCode,ShipCountry,ShippingTime,CompanyName_y,Phone
0,1,Chai,1,1,18.0,Beverages,Exotic Liquids,Charlotte Cooper,Purchasing Manager,London,...,2,76.83,QUICK-Stop,Cunewalde,NaN,01307,Germany,6.0,United Package,(503) 555-3199
1,1,Chai,1,1,18.0,Beverages,Exotic Liquids,Charlotte Cooper,Purchasing Manager,London,...,2,147.26,Rattlesnake Canyon Grocery,Albuquerque,NM,87110,USA,6.0,United Package,(503) 555-3199
2,1,Chai,1,1,18.0,Beverages,Exotic Liquids,Charlotte Cooper,Purchasing Manager,London,...,1,12.69,Lonesome Pine Restaurant,Portland,OR,97219,USA,10.0,Speedy Express,(503) 555-9831


In [68]:
suppliers_analysis["Revenue"] = suppliers_analysis["DiscountedUnitPrice"]*suppliers_analysis["Quantity"]

suppliers_analysis.head(3)

,ProductID,ProductName,SupplierID,CategoryID,UndiscountedUnitPrice,CategoryName,CompanyName_x,ContactName,ContactTitle,City,...,Freight,ShipName,ShipCity,ShipRegion,ShipPostalCode,ShipCountry,ShippingTime,CompanyName_y,Phone,Revenue
0,1,Chai,1,1,18.0,Beverages,Exotic Liquids,Charlotte Cooper,Purchasing Manager,London,...,76.83,QUICK-Stop,Cunewalde,NaN,01307,Germany,6.0,United Package,(503) 555-3199,648.0
1,1,Chai,1,1,18.0,Beverages,Exotic Liquids,Charlotte Cooper,Purchasing Manager,London,...,147.26,Rattlesnake Canyon Grocery,Albuquerque,NM,87110,USA,6.0,United Package,(503) 555-3199,259.2
2,1,Chai,1,1,18.0,Beverages,Exotic Liquids,Charlotte Cooper,Purchasing Manager,London,...,12.69,Lonesome Pine Restaurant,Portland,OR,97219,USA,10.0,Speedy Express,(503) 555-9831,288.0


## Analysis

### **Suppliers by Total Revenue & Total Quantity Sold**
**Objective**
- Identify the suppliers who contribute the most in the revenue and supply the most and quantities of products.


**Business Use-case**

This analysis helps in understanding the distribution of product supply across different suppliers. It aids in identifying key suppliers and optimizing supplier relationships.

## **Supplier Reliability Analysis**
**Objectives:**
- Analyze the average shipping time for each supplier.
- Identify the most reliable suppliers based on shipping time.

**Business Use-case**: 

This analysis helps in evaluating supplier performance in terms of timely deliveries, which is crucial for maintaining the supply chain's efficiency.
